In [5]:
#import libraries
import random
import sqlite3
from sqlite3 import Error

#define shopping listS
shoppinglist = ['bread','milk','eggs','chicken','beef','pork','tofu','noodles','rice','water','juice','milk','coffee','tea','lettuce','watermelon','bananas','tomatoes','pickles','lemons','peas','cabbage','chili','cashews','edamame','carrots','radish','beer','papertowels','napkins']
len(shoppinglist)

30

In [ ]:
#generate 20 transactions
def generate_transactions(inputlist):
    index = 0
    transactions = []
    while index < 20:
        
        #generate random number of items in each shopping transaction
        transnum = random.randrange(1,20,1)

        #generate transactions using the transnum
        transaction = random.sample(inputlist,transnum)

        #sort alphabetically and append to transactions list
        transaction.sort()
        transactions.append(transaction)

        index +=1
    
    return transactions

#show transactions
transactions = generate_transactions(shoppinglist)
for listitem in range(len(transactions)):
    print(transactions[listitem])
    if 'cabbage' in transactions[listitem]:
        if 'lettuce' in transactions[listitem]:
            print(True)

In [1]:
#create database(s) and relevant tables

def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
    return conn

def create_table(conn,name):
    """ create table from the create_table_statement"""
    try:
        c = conn.cursor()
        c.execute("""CREATE TABLE IF NOT EXISTS %s (item_id text PRIMARY KEY)""" %(name))
        print('Table created')
        conn.commit()
        conn.close()
    except Error as e:
        print(e)
    

def populate_table(conn,tablename,itemname):
    """populate table from populate_statement"""
    try:
        c = conn.cursor()
        c.execute("""INSERT INTO %s (item_id) VALUES (?)""" %(tablename),[itemname])
        print('Table populated')
        conn.commit()
        conn.close()
    except Error as e:
        print(e)


def query(conn,statement):
    try:
        c = conn.cursor()
        c.execute(statement)
        conn.commit()
        row = c.fetchall()
        print(row)
        conn.close()
    except Error as e:
        print(e)
        
def main():
    database = r"C:\Users\Nick\CS634_Data_Mining\midtermproject.db"
    
    index = 0
    #create the 20 transaction tables
    for index in range(len(transactions)):
        #create database connection
        conn = create_connection(database)
        if conn is not None:
            table_name = 'transaction_'+str(index)
            create_table(conn,table_name)
            #populate each table with the items in each respective transaction
            for item in transactions[index]:
                item_name = item
                conn = create_connection(database)
                populate_table(conn,table_name,item_name)
        else:
            print("Cannot connect to database")
            
#test
if __name__ == '__main__':
    #create_connection(r"C:\Users\Nick\CS634_Data_Mining\midtermproject.db")
    #main()
    statement = "SELECT * FROM transaction_4 ORDER BY item_id ASC"
    #statement = "UPDATE transaction_4 SET item_id = 'pork' WHERE item_id='edamame'"
    query(create_connection(r"C:\Users\Nick\CS634_Data_Mining\midtermproject.db"),statement)


NameError: name 'Error' is not defined

In [7]:
## Initialize the algorithm

In [264]:
#get transactions from database in list form
def getItems(conn,transnum):
    try:
        items_list = []
        c = conn.cursor()
        c.execute("SELECT * FROM %s"%('transaction_'+str(transnum)))
        transaction = c.fetchall()
        for item in transaction:
            items_list.append(item[0])
        return items_list
        conn.close()
    except Error as e:
        print(e)
   

#create hash map with collision prevention and visited flags
def createHashMap(transaction):
    hash_map = [[] for i in range(100)]
    for item in transaction:
        
        #hash item value
        h = 0
        for char in item:
            h += ord(char)
            h = h % 50
        key = h
        if hash_map[key] is None:
            hash_map[key].append(item)
        else:
            hash_map[key].append(item)
    return hash_map


#get hashes of all items in list to use for finding the support
def hashShoppingList(shoplist):
    hashlist = []
    for item in range(len(shoplist)):
        hashlist.append(getHash(shoplist[item]))
    return hashlist

        
                
#test
conn = create_connection(r"C:\Users\Nick\CS634_Data_Mining\midtermproject.db")
result_list = getItems(conn,0)
hash_map = createHashMap(result_list)
items_hash = hashShoppingList(shoppinglist)

print(hash_map)
print(items_hash)


2.6.0


NameError: name 'getHash' is not defined

In [272]:
def generateGroups(input_set,size):
    input_set = list(input_set)
    if size == 0:
        return [[]]
    new_set = []
    for index in range(0,len(input_set)):
        item = input_set[index]
        remaining = input_set[index + 1:]
        for j in generateGroups(remaining,size-1):
            new_set.append([item]+list(j))
    return new_set

def unionSets(input_1,input_2):
    return set(input_1).union(set(input_2))

def getItems(conn,transnum):
    try:
        items_list = []
        c = conn.cursor()
        c.execute("SELECT item_id FROM %s ORDER BY item_id ASC"%('transaction_'+str(transnum)))
        transaction = c.fetchall()
        for item in transaction:
            items_list.append(item[0])
        return items_list
        conn.close()
    except Error as e:
        print(e)

def getTransactions(conn,translength):
        transactions = []
        for i in range(translength):
            transaction = getItems(conn,i)
            transactions.append(transaction)
        return transactions

def scanTransactions(itemset,transactions):
    count = 0
    for i in range(len(transactions)):
        if set(itemset).issubset(transactions[i]):
            count +=1
    return count   

def getUnions(set1,n):
    sets = []
    for item in set1:
        if type(item) == type(''):
            item = [item]
            for i in set1:
                if type(i) == type(''):
                    i = [i]
                    if item != i:
                        u = set(item).union(set(i))
                        if u not in sets and len(u) == n:
                            sets.append(u)
        else:
            itemset = list(item)
            #print(set(itemset))
            for j in set1:
                if type(j) != type((1,2)):
                    u = set(itemset).union({j})
                    if u not in sets and len(u) == n:
                        sets.append(u)
                else:
                    itemset2 = set(list(j))
                    u = itemset2.union(itemset)
                    if u not in sets and len(u) == n:
                        sets.append(u)
                      
    return sets

def apriori(set_list,min_support):
    #initialize
    discarded = {}
    C = {}
    L = {}
    stop = False
    n = 1
    while stop == False:
        if n <= 1:
            for transaction in range(len(set_list)):
                for item in set_list[transaction]:
                    #prevent duplicate scans of same subset
                    if item not in C:
                        #prevent scans of already discarded items
                        if item not in discarded:
                            sup_count = scanTransactions({item},set_list)
                            C[item] = sup_count

            #print first scan results
            print('\nIteration {}'.format(n))
            print('C1 candidate set: \n')
            
            for i in C:
                print(i,':',C[i])
            #create L with items that meet minimum support requirement
            for item in C:
                sup_calc = C[item] / len(set_list)
                if item not in discarded:
                    if sup_calc >= min_support:
                        L[item] = C[item]
                    else:
                        discarded[item] = sup_calc
                    
            print('\nL1 set: \n')
            for i in L:
                print(i,':',L[i])
            print('#'*50)
            
            L_n = L #helps to generate sets for second iteration 
            n += 1
            if L is None:
                stop = True
                break
        elif n > 1:
            C_n = {}
            #union of the sets
            sets = []
            L_n = list(L_n)
            sets = getUnions(L_n,n)
            
            #save last iteration's results to a variable
            L_n_minus_1 = L_n
            
            #reinitialize L_n for the following iteration
            L_n = {}
            
            for i in range(len(sets)):
                support_count = scanTransactions(sets[i],set_list)
                C_n[tuple(sets[i])] = support_count
            if C_n == {}:
                stop = True
                break
            
            #print candidate set
            print('\nIteration {}'.format(n))
            print('C{} candidate set: \n'.format(n))
            for i in C_n:
                print(i,':',C_n[i])
            
            #calculate supports of each item in candidate set and prune
            for item in C_n:
                if item not in discarded:
                    sup_calc = C_n[item] / len(set_list)
                    if sup_calc >= min_support:
                        L_n[item] = C_n[item]
                    else:
                        discarded[item] = sup_calc
             
            #print passing set
            print('\nL{} set: \n'.format(n))
            for i in L_n:
                print(i,':',L_n[i])
            print('#'*50)
            
            if L_n == {} or L_n is None:
                stop = True
                return L_n_minus_1
                break
            n += 1
    return L_n_minus_1
            
def makeAssociationRules(inputset,min_confidence,transactions):
    print("Association Rules that meet minimum confidence: \n")
    for i in inputset:
        i = list(i)
        for j in i:
            right = {j}
            left = set(i) - right
            both = i
            counts_both = scanTransactions(both,transactions)
            counts_left = scanTransactions(list(left),transactions)
            sup_both = counts_both / len(transactions)
            sup_left = counts_left / len(transactions)
            if sup_left != 0:
                if sup_both / sup_left >= min_confidence:
                    print(left,'->',right,'    ','confidence: ',(sup_both/sup_left),'\n')
            
conn = create_connection(r"C:\Users\Nick\CS634_Data_Mining\midtermproject.db")
transactionsList = getTransactions(conn,20)
results = apriori(transactionsList,0.3)
makeAssociationRules(results,.6,transactionsList)
#TODO: checkout generating association rules when only L_n is iteration 1 results

2.6.0

Iteration 1
C1 candidate set: 

bananas : 6
cabbage : 13
carrots : 8
cashews : 5
juice : 6
milk : 12
pork : 11
radish : 6
rice : 5
watermelon : 6
beer : 8
chili : 5
lettuce : 11
noodles : 9
pickles : 7
tea : 10
tomatoes : 4
bread : 9
lemons : 6
napkins : 7
papertowels : 8
tofu : 6
chicken : 9
edamame : 5
water : 6
coffee : 5
peas : 9
beef : 6
eggs : 6

L1 set: 

bananas : 6
cabbage : 13
carrots : 8
juice : 6
milk : 12
pork : 11
radish : 6
watermelon : 6
beer : 8
lettuce : 11
noodles : 9
pickles : 7
tea : 10
bread : 9
lemons : 6
napkins : 7
papertowels : 8
tofu : 6
chicken : 9
water : 6
peas : 9
beef : 6
eggs : 6
##################################################

Iteration 2
C2 candidate set: 

('cabbage', 'bananas') : 3
('bananas', 'carrots') : 4
('bananas', 'juice') : 2
('bananas', 'milk') : 4
('bananas', 'pork') : 4
('radish', 'bananas') : 4
('watermelon', 'bananas') : 2
('bananas', 'beer') : 3
('lettuce', 'bananas') : 2
('noodles', 'bananas') : 3
('pickles', 'bananas') : 2
(